# Sudoku

I recently started solving some sudoku puzzles for fun,
and the thought occured to me: *Why not try to write a sudoku solver-generator in Python?*

In [ ]:
import networkx as nx
import rich
from rich.table import Table
import itertools
import matplotlib.pyplot as plt


def gen_sudoku():
    sudoku = nx.sudoku_graph()
    mapping = {}
    for n in sudoku.nodes:
        row_index = n // 9
        col_index = n - (row_index * 9)
        mapping[n] = ((row_index, col_index))
    sudoku = nx.relabel_nodes(sudoku, mapping)
    for n in sudoku.nodes:
        sudoku.nodes[n]["value"] = 0
    return sudoku

sudoku = gen_sudoku()

In [ ]:
def plot_sudoku(sudoku, show_node_labels=False):
    pos = {}
    for n in sudoku.nodes:
        row, col = n
        y = 9 - row
        x = col
        pos[n] = (x, y)
    if show_node_labels:
        labels = None
    else:
        labels = {n:sudoku.nodes[n]["value"] for n in sudoku.nodes}
    nx.draw(sudoku, pos=pos, labels=labels, with_labels=True, node_color=[[0.8, 0.8, 0.8]])
    plt.show()
    
plot_sudoku(sudoku)

In [ ]:
plot_sudoku(sudoku, show_node_labels=True)

In [ ]:
def get_successor_nodes(sudoku, *source_nodes):
    successor_nodes = set()
    for sn in source_nodes:
        bfs_succ = dict(nx.bfs_successors(sudoku, sn, depth_limit=1))
        for u, vs in bfs_succ.items():
            successor_nodes.add(u)
            [successor_nodes.add(v) for v in vs]
        
    return list(successor_nodes)

def plot_sub_sudoku(sudoku, *source_nodes):
    successor_nodes = get_successor_nodes(sudoku, *source_nodes)
    plot_sudoku(sudoku.subgraph(successor_nodes))
    
plot_sub_sudoku(sudoku, (0,0))

In [ ]:
def gen_sudoku_with_values(values):
    sudoku = gen_sudoku()
    for i, row in enumerate(values):
        for j, val in enumerate(row):
            sudoku.nodes[(i,j)]["value"] = val
    return sudoku
            
easy_sudoku = gen_sudoku_with_values([
    [4,0,0,2,0,5,8,6,0],
    [0,0,0,0,0,8,4,9,3],
    [6,0,0,0,0,0,0,2,7],
    [1,0,0,0,6,9,0,5,0],
    [3,0,0,0,8,0,0,0,9],
    [0,5,0,4,3,0,0,0,2],
    [8,6,0,0,0,0,0,0,4],
    [7,2,1,8,0,0,0,0,0],
    [0,9,4,1,0,3,0,0,6]
])
plot_sudoku(easy_sudoku)

In [ ]:
def get_nodes_with_value(sudoku, value):
    nodes = []
    for n in sudoku.nodes:
        if value == sudoku.nodes[n]["value"]:
            nodes.append(n)
    return nodes

nodes_with_4 = get_nodes_with_value(easy_sudoku, 4)
nodes_with_4

In [ ]:
plot_sub_sudoku(easy_sudoku, *nodes_with_4)

In [ ]:
def solve(sudoku):
    new_sudoku = sudoku.copy()
    possible_values = set(range(1, 10))
    cannot_be = {n:set() for n in new_sudoku.nodes}
    
    def update():
        for value in possible_values:
            nodes_with_value = get_nodes_with_value(new_sudoku, value)
            successor_nodes = get_successor_nodes(new_sudoku, *nodes_with_value)
            for sn in successor_nodes:
                cannot_be[sn].add(value)
        for n, vals in cannot_be.items():
            if len(vals) == 8 and new_sudoku.nodes[n]["value"] == 0:
                new_sudoku.nodes[n]["value"] = list(possible_values - vals)[0]
                    
    def is_sudoku_solved():
        return all(new_sudoku.nodes[n]["value"] != 0 for n in new_sudoku.nodes)
    
    while not is_sudoku_solved():
        update()
        
    return new_sudoku

plot_sudoku(solve(easy_sudoku))